<a href="https://colab.research.google.com/github/xbkaishui/cv_tools_box/blob/main/Kornia_Augmentations_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install git+https://github.com/kornia/kornia git+https://github.com/albumentations-team/albumentations

In [ ]:
import timeit
import os
import numpy as np
import pandas as pd
import torch


general_setup = lambda batch_size, img_size: f"""
import torch
import kornia
from torchvision.transforms import transforms
import albumentations.augmentations as A
in_tensor = torch.randn({batch_size}, 3, {img_size}, {img_size}).to('cuda:0')
in_pil = transforms.ToPILImage()(in_tensor.cpu()[0])
"""

perspective_setup = """
tv_fn = transforms.RandomPerspective(p=1.0)
kor_fn = kornia.augmentation.RandomPerspective(p=1.0)
alb_fn = A.geometric.transforms.Perspective(p=1.)
"""

colorjitter_setup = """
tv_fn = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2)
kor_fn = kornia.augmentation.ColorJitter(0.5, 0.5, 0.5, 0.5)
alb_fn = A.transforms.ColorJitter(0.5, 0.5, 0.5, 0.5, p=1.)
"""

affine_setup = """
tv_fn = transforms.RandomAffine(360, (0.1, 0.1), (1.1, 1.2), (0.1, 0.1), resample=2)
kor_fn = kornia.augmentation.RandomAffine(360, (0.1, 0.1), (1.1, 1.2), (0.1, 0.1))
alb_fn = A.geometric.transforms.Affine(scale=(1.1, 1.2), translate_percent=0.1, rotate=360, shear=45, p=1.)
"""

vflip_setup = """
tv_fn = transforms.RandomVerticalFlip(p=1.0)
kor_fn = kornia.augmentation.RandomVerticalFlip(p=1.0)
alb_fn = A.transforms.VerticalFlip(p=1.)
"""

hflip_setup = """
tv_fn = transforms.RandomHorizontalFlip(p=1.0)
kor_fn = kornia.augmentation.RandomHorizontalFlip(p=1.0)
alb_fn = A.transforms.HorizontalFlip(p=1.)
"""

rotate_setup = """
tv_fn = transforms.RandomRotation(180, resample=2)
kor_fn = kornia.augmentation.RandomRotation(180.)
alb_fn = A.geometric.rotate.Rotate(180., p=1.)
"""

crop_setup = """
tv_fn = transforms.RandomCrop((180, 180))
kor_fn = kornia.augmentation.RandomCrop((180, 180))
alb_fn = A.crops.transforms.RandomCrop(180, 180, p=1.)
"""

erasing_setup = """
tv_fn = transforms.RandomErasing(p=1.0)
kor_fn = kornia.augmentation.RandomErasing(p=1.0, scale=(0.02, 0.33), ratio= (0.3, 3.3))
"""

grayscale_setup = """
tv_fn = transforms.RandomGrayscale(p=1.0)
kor_fn = kornia.augmentation.RandomGrayscale(p=1.0)
alb_fn = A.ToGray(p=1.)
"""

res_crop_setup = """
tv_fn = transforms.RandomResizedCrop((180, 180))
kor_fn = kornia.augmentation.RandomResizedCrop((180, 180), (0.08, 1.0), (0.75, 1.33))
alb_fn = A.crops.transforms.RandomResizedCrop(180, 180, (0.08, 1.0), (0.75, 1.33), p=1.)
"""

cent_crop_setup = """
tv_fn = transforms.CenterCrop((180, 180))
kor_fn = kornia.augmentation.CenterCrop((180, 180))
alb_fn = A.crops.transforms.CenterCrop(180, 180, p=1.)
"""


if __name__ == '__main__':
    save_to = "data"
    num = 10

    image_sizes = {
        # Coefficients:   size
        'efficientnet-b0': 224,
        # 'efficientnet-b1': 240,
        # 'efficientnet-b2': 260,
        # 'efficientnet-b3': 300,
        # 'efficientnet-b4': 380,
        # 'efficientnet-b5': 456,
        # 'efficientnet-b6': 528,
        # 'efficientnet-b7': 600,
    }
    fn_set_ups = {
        'RandomPerspective': perspective_setup,
        'ColorJitter': colorjitter_setup,
        'RandomAffine': affine_setup,
        'RandomVerticalFlip': vflip_setup,
        'RandomHorizontalFlip': hflip_setup,
        'RandomRotate': rotate_setup,
        'RandomCrop': crop_setup,
        'RandomErasing': erasing_setup,
        'RandomGrayscale': grayscale_setup,
        'RandomResizedCrop': res_crop_setup,
        'RandomCenterCrop': cent_crop_setup
    }

    tv_timer = lambda batch_size, net, name: timeit.Timer(
        f"[tv_fn({'in_tensor[0].squeeze()' if name == 'RandomErasing' else 'in_pil'}) for _ in range({batch_size * num})]" ,
        setup=general_setup(batch_size, image_sizes[net]) + fn_set_ups[name]
    )

    alb_timer = lambda batch_size, net, name: timeit.Timer(
        f"[alb_fn({'in_tensor[0].squeeze().cpu().numpy().data'}) for _ in range({batch_size * num})]" ,
        setup=general_setup(batch_size, image_sizes[net]) + fn_set_ups[name]
    )

    kor_timer = lambda batch_size, net, name: timeit.Timer(
        f"[kor_fn(in_tensor) for _ in range({num})]",
        setup=general_setup(batch_size, image_sizes[net]) + fn_set_ups[name]
    )

    try:
        os.mkdir(save_to)
    except:
        pass

    for timer_name, batch_size, timer in [
          ("torchvision", 1, tv_timer),
          ("albumentations", 1, alb_timer),
          ("kornia", 1, kor_timer),
          ("kornia", 32, kor_timer),
          ("kornia", 128, kor_timer),
    ]:
        dfs = []
        for name in fn_set_ups:
            row_res = {'op_name': name}
            for net in image_sizes:
                timer = kor_timer(batch_size, net, name)
                timer_res = timer.repeat(num, 1)

                row_res.update({net: np.mean(timer_res) / num / batch_size * 1000})
                print(timer_name, name, net, "image_size=%d" % image_sizes[net], "batchsize=%d" % batch_size, "%.2f±%.2fms" % (np.mean(timer_res) / num / batch_size * 1000, np.std(timer_res) * 1000))
            dfs.append(pd.DataFrame.from_dict(row_res, orient='index').T)
        df = pd.concat(dfs)
        df.to_csv(f"{save_to}/{timer_name}_bs{batch_size}.csv", index=None)

        with torch.no_grad():
            torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torchvision RandomPerspective efficientnet-b0 image_size=224 batchsize=1 9.13±125.09ms
torchvision ColorJitter efficientnet-b0 image_size=224 batchsize=1 4.77±7.26ms


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1362: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


torchvision RandomAffine efficientnet-b0 image_size=224 batchsize=1 3.02±5.61ms
torchvision RandomVerticalFlip efficientnet-b0 image_size=224 batchsize=1 0.35±0.40ms
torchvision RandomHorizontalFlip efficientnet-b0 image_size=224 batchsize=1 0.35±0.63ms


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1249: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


torchvision RandomRotate efficientnet-b0 image_size=224 batchsize=1 2.00±5.02ms
torchvision RandomCrop efficientnet-b0 image_size=224 batchsize=1 4.37±6.20ms
torchvision RandomErasing efficientnet-b0 image_size=224 batchsize=1 2.68±6.66ms
torchvision RandomGrayscale efficientnet-b0 image_size=224 batchsize=1 0.49±0.86ms
torchvision RandomResizedCrop efficientnet-b0 image_size=224 batchsize=1 4.83±10.45ms
torchvision RandomCenterCrop efficientnet-b0 image_size=224 batchsize=1 2.98±0.64ms
albumentations RandomPerspective efficientnet-b0 image_size=224 batchsize=1 5.31±5.38ms
albumentations ColorJitter efficientnet-b0 image_size=224 batchsize=1 4.32±2.97ms
albumentations RandomAffine efficientnet-b0 image_size=224 batchsize=1 3.57±2.90ms
albumentations RandomVerticalFlip efficientnet-b0 image_size=224 batchsize=1 0.34±0.23ms
albumentations RandomHorizontalFlip efficientnet-b0 image_size=224 batchsize=1 0.33±0.11ms
albumentations RandomRotate efficientnet-b0 image_size=224 batchsize=1 1.73